Import the data

In [2]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1, as_frame=False)
print(mnist.keys())

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])


Train/Validation/Test Split

In [3]:
X, y = mnist["data"], mnist["target"]
X_train, X_val, X_test = X[:50000], X[50000:60000], X[-10000:]  # The training set is inherently split (and stratified)
y_train, y_val, y_test = y[:50000], y[50000:60000], y[-10000:]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(50000, 784)
(10000, 784)
(10000, 784)
(50000,)
(10000,)
(10000,)


In [4]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import reciprocal, uniform
from sklearn.svm import SVC

svm_clf = SVC(kernel="rbf", gamma="scale", random_state=256)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ....C=7.009325797765703, gamma=0.008688476780448094; total time=   0.2s
[CV] END ....C=7.009325797765703, gamma=0.008688476780448094; total time=   0.2s
[CV] END ....C=7.009325797765703, gamma=0.008688476780448094; total time=   0.2s
[CV] END ....C=3.535886658776003, gamma=0.002055428188115425; total time=   0.2s
[CV] END ....C=3.535886658776003, gamma=0.002055428188115425; total time=   0.2s
[CV] END ....C=3.535886658776003, gamma=0.002055428188115425; total time=   0.2s
[CV] END ....C=1.0148154538163392, gamma=0.06690912128026312; total time=   0.2s
[CV] END ....C=1.0148154538163392, gamma=0.06690912128026312; total time=   0.3s
[CV] END ....C=1.0148154538163392, gamma=0.06690912128026312; total time=   0.2s
[CV] END .....C=10.95439523349186, gamma=0.01173039938888608; total time=   0.3s
[CV] END .....C=10.95439523349186, gamma=0.01173039938888608; total time=   0.2s
[CV] END .....C=10.95439523349186, gamma=0.01173

RandomizedSearchCV(cv=3, estimator=SVC(random_state=256),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x139b12d30>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x139bb8e80>},
                   verbose=2)

SVM (Gaussian RBF Kernel) Classifier

In [6]:
import time

start_time = time.time()

rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

end_time = time.time()
print("SVC Training Complete. Time Elapsed: %f seconds" % (end_time - start_time))

SVC Training Complete. Time Elapsed: 325.916603 seconds


Random Forest and Extra Trees Classifiers

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

start_time = time.time()

random_forest_clf = RandomForestClassifier(n_estimators=200, random_state=42)
random_forest_clf.fit(X_train, y_train)

end_time = time.time()
print("Random Forest Training Complete. Time Elapsed: %f seconds" % (end_time - start_time))

start_time = time.time()

extra_trees_clf = ExtraTreesClassifier(n_estimators=200, random_state=42)
extra_trees_clf.fit(X_train, y_train)

end_time = time.time()
print("Extra Trees Training Complete. Time Elapsed: %f seconds" % (end_time - start_time))

Random Forest Training Complete. Time Elapsed: 62.555339 seconds


Multilayer Perceptron Classifier

In [7]:
from sklearn.neural_network import MLPClassifier

start_time = time.time()

mlp_clf = MLPClassifier(random_state=42)
mlp_clf.fit(X_train, y_train)

end_time = time.time()
print("MLP Training Complete. Time Elapsed: %f seconds" % (end_time - start_time))

MLP Training Complete. Time Elapsed: 97.766538 seconds


SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

start_time = time.time()

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

end_time = time.time()
print("SGD Training Complete. Time Elapsed: %f seconds" % (end_time - start_time))

Validation accuracy of each classifier

In [ ]:
estimators = [svm_clf, random_forest_clf, extra_trees_clf, mlp_clf, sgd_clf]
[estimator.score(X_val, y_val) for estimator in estimators]